<a href="https://colab.research.google.com/github/pxs1990/regression_analysis_project/blob/main/complete_regression_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is done on synthetic-randomly generated- data and might not make much sense on the prediction vs features relationship. However, this project gives you a hndson experience on several regression algorithms and gives you a very good insights about how to choose a best algorithms based on model evaluation metrics.

In [21]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error

In [22]:
# Generate synthetic data
np.random.seed(42)
X, y = make_regression(n_samples=1000, n_features=5, noise=0.2, bias=3.0)#y_synthetic=y_true + GaussianNoise(0,0.2)=68% betn (-0.2,0.2)
X

array([[ 2.05654356,  0.60685059,  0.48268789, -1.13088844,  0.42009449],
       [-0.79919201, -0.64596418, -0.18289644, -0.48274352,  1.37487642],
       [ 1.07600714, -0.79602586, -0.75196933,  0.02131165, -0.31905394],
       ...,
       [ 0.49968511,  0.2394045 ,  1.48724616,  0.47200227, -0.58005324],
       [-0.64148691,  0.01914778, -0.66198218,  0.48787228,  0.42588721],
       [ 0.91539028, -0.83305606, -1.77624633, -0.54954027, -0.08059975]])

In [ ]:
# np.random.seed(0)#reproduce same no
# np.random.randint(2)

In [23]:
# Categorical data
cat_data = np.random.choice(['A', 'B', 'C'], size=1000, replace=True).reshape(-1, 1)#replace=True argument ensures that sampling is done with replacement
X = np.concatenate([X, cat_data], axis=1)
X

array([['2.0565435648295414', '0.606850593215671', '0.4826878872092127',
        '-1.1308884353259157', '0.4200944904518413', 'C'],
       ['-0.7991920066502536', '-0.6459641791245353',
        '-0.18289644110484016', '-0.4827435221820324',
        '1.3748764191481848', 'A'],
       ['1.0760071385985983', '-0.796025856803489',
        '-0.7519693317388191', '0.02131164867194222',
        '-0.31905393566661816', 'B'],
       ...,
       ['0.49968511030293383', '0.23940450053537707',
        '1.4872461553359273', '0.4720022709192744', '-0.580053241370043',
        'C'],
       ['-0.6414869090443096', '0.01914777741575206',
        '-0.6619821777966091', '0.487872284871074',
        '0.42588720619303577', 'B'],
       ['0.9153902755601617', '-0.8330560570007587',
        '-1.7762463334664518', '-0.5495402745955349',
        '-0.08059974929676088', 'C']], dtype='<U32')

In [24]:
# Introduce missing values(thru masking)
missing_mask = np.random.rand(*X.shape) < 0.1# * unpacks tuple. e.g. *(2,3)=2,3
X[missing_mask] = np.nan
X

array([['2.0565435648295414', '0.606850593215671', '0.4826878872092127',
        '-1.1308884353259157', '0.4200944904518413', 'C'],
       ['-0.7991920066502536', '-0.6459641791245353',
        '-0.18289644110484016', 'nan', '1.3748764191481848', 'A'],
       ['1.0760071385985983', 'nan', '-0.7519693317388191',
        '0.02131164867194222', '-0.31905393566661816', 'B'],
       ...,
       ['0.49968511030293383', '0.23940450053537707',
        '1.4872461553359273', '0.4720022709192744', '-0.580053241370043',
        'C'],
       ['-0.6414869090443096', '0.01914777741575206',
        '-0.6619821777966091', '0.487872284871074',
        '0.42588720619303577', 'nan'],
       ['nan', '-0.8330560570007587', '-1.7762463334664518',
        '-0.5495402745955349', '-0.08059974929676088', 'C']], dtype='<U32')

In [25]:
# Introduce outliers
X[0:10,0] = 1000  # 10 Outliers in first feature
X

array([['1000', '0.606850593215671', '0.4826878872092127',
        '-1.1308884353259157', '0.4200944904518413', 'C'],
       ['1000', '-0.6459641791245353', '-0.18289644110484016', 'nan',
        '1.3748764191481848', 'A'],
       ['1000', 'nan', '-0.7519693317388191', '0.02131164867194222',
        '-0.31905393566661816', 'B'],
       ...,
       ['0.49968511030293383', '0.23940450053537707',
        '1.4872461553359273', '0.4720022709192744', '-0.580053241370043',
        'C'],
       ['-0.6414869090443096', '0.01914777741575206',
        '-0.6619821777966091', '0.487872284871074',
        '0.42588720619303577', 'nan'],
       ['nan', '-0.8330560570007587', '-1.7762463334664518',
        '-0.5495402745955349', '-0.08059974929676088', 'C']], dtype='<U32')

In [26]:
# Convert feature and target arrays to DataFrame
columns = ['num1', 'num2', 'num3', 'num4', 'num5', 'cat']
df= pd.DataFrame(X, columns=columns)
df['target'] = y

In [27]:
df.head()

,num1,num2,num3,num4,num5,cat,target
0,1000,0.606850593215671,0.4826878872092127,-1.1308884353259157,0.4200944904518413,C,77.863853
1,1000,-0.6459641791245353,-0.18289644110484016,nan,1.3748764191481848,A,-38.750140
2,1000,nan,-0.7519693317388191,0.02131164867194222,-0.31905393566661816,B,-21.312337
3,1000,-0.828497024780246,1.4898634386020821,-1.6431888111541673,-1.6009039784765156,A,-84.058551
4,1000,0.5032520647495048,-0.645572072035395,-0.3173504922425348,-1.6610834155671725,A,-83.094538


In [28]:
df.dtypes

num1       object
num2       object
num3       object
num4       object
num5       object
cat        object
target    float64
dtype: object

# **Data Preprocessing**

In [29]:
X=df.drop(columns=['target'])
X.head()

,num1,num2,num3,num4,num5,cat
0,1000,0.606850593215671,0.4826878872092127,-1.1308884353259157,0.4200944904518413,C
1,1000,-0.6459641791245353,-0.18289644110484016,nan,1.3748764191481848,A
2,1000,nan,-0.7519693317388191,0.02131164867194222,-0.31905393566661816,B
3,1000,-0.828497024780246,1.4898634386020821,-1.6431888111541673,-1.6009039784765156,A
4,1000,0.5032520647495048,-0.645572072035395,-0.3173504922425348,-1.6610834155671725,A


In [30]:
y=df['target']
y

0      77.863853
1     -38.750140
2     -21.312337
3     -84.058551
4     -83.094538
         ...    
995     6.582149
996    63.820568
997    54.076443
998    -5.484750
999   -54.188761
Name: target, Length: 1000, dtype: float64

In [31]:
numeric_columns = ['num1', 'num2', 'num3', 'num4', 'num5']
X[numeric_columns] = X[numeric_columns].apply(pd.to_numeric, errors='coerce')# coerse=nan if error occured.
X.dtypes

num1    float64
num2    float64
num3    float64
num4    float64
num5    float64
cat      object
dtype: object

In [32]:
# Select numeric columns for imputation
numeric_cols = X.select_dtypes(include='number').columns
X_numeric = X[numeric_cols]

# Select categorical columns for imputation
categorical_cols = X.select_dtypes(exclude='number').columns
X_categorical = X[categorical_cols]

In [33]:
X_numeric.head()

,num1,num2,num3,num4,num5
0,1000.0,0.606851,0.482688,-1.130888,0.420094
1,1000.0,-0.645964,-0.182896,NaN,1.374876
2,1000.0,NaN,-0.751969,0.021312,-0.319054
3,1000.0,-0.828497,1.489863,-1.643189,-1.600904
4,1000.0,0.503252,-0.645572,-0.317350,-1.661083


In [34]:
X_categorical.head()

,cat
0,C
1,A
2,B
3,A
4,A


In [35]:
from sklearn.impute import SimpleImputer
# Impute missing values for numeric features
numeric_imputer = SimpleImputer(strategy='mean')
X_numeric_imputed = numeric_imputer.fit_transform(X_numeric)
X_numeric_imputed = pd.DataFrame(X_numeric_imputed, columns=numeric_cols, index=X_numeric.index)

# Impute missing values for categorical features
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_imputed = categorical_imputer.fit_transform(X_categorical)
X_categorical_imputed = pd.DataFrame(X_categorical_imputed, columns=categorical_cols, index=X_categorical.index)

# Concatenate numeric and categorical features
X_imputed = pd.concat([X_numeric_imputed, X_categorical_imputed], axis=1)


In [36]:
X_imputed.isna().sum()

num1    0
num2    0
num3    0
num4    0
num5    0
cat     0
dtype: int64

In [37]:
X_imputed.head(10)

,num1,num2,num3,num4,num5,cat
0,1000.0,0.606851,0.482688,-1.130888,0.420094,C
1,1000.0,-0.645964,-0.182896,0.038078,1.374876,A
2,1000.0,0.001516,-0.751969,0.021312,-0.319054,B
3,1000.0,-0.828497,1.489863,-1.643189,-1.600904,A
4,1000.0,0.503252,-0.645572,-0.317350,-1.661083,A
5,1000.0,-0.007045,-0.451949,-0.371858,-0.789339,B
6,1000.0,0.120031,0.436324,-1.022793,0.038003,C
7,1000.0,0.553040,1.818062,-0.248529,0.775155,A
8,1000.0,0.140983,0.145521,0.695402,-0.614123,C
9,1000.0,-0.044893,-0.399899,0.038078,0.269295,C


In [38]:
y.isna().sum()

0

In [39]:
# One-hot encode categorical data
cat_cols = ['cat']
num_cols = [c for c in X_imputed.columns if c != 'cat']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols)
    ])

X_processed = preprocessor.fit_transform(X_imputed)

In [40]:
X_processed

array([[1.        , 0.60189627, 0.51476495, ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.40593956, 0.42097661, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.50721395, 0.34078781, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.0034968 , 0.54442287, 0.65631855, ..., 0.        , 1.        ,
        0.        ],
       [0.00235905, 0.50997182, 0.35346802, ..., 0.        , 0.        ,
        1.        ],
       [0.01401855, 0.37667593, 0.19645562, ..., 0.        , 1.        ,
        0.        ]])

# **Winsorizing extreme 10% of the data to handle outliers:**
winsorize at the 10th and 90th percentiles, 10% of the lowest values by the value at 10th percentile and 10% of the highest values by the value at 90th percentile.

In [41]:
# Assuming X is a NumPy array
from scipy.stats.mstats import winsorize

# Specify the limits for winsorization
lower_limit = 0.101
upper_limit = 0.05

# Perform winsorization on the first column of array X
X_processed[:, 0] = winsorize(X_processed[:, 0], limits=(lower_limit, upper_limit))


In [42]:
X_processed

array([[0.01401855, 0.60189627, 0.51476495, ..., 0.        , 1.        ,
        0.        ],
       [0.01401855, 0.40593956, 0.42097661, ..., 0.        , 0.        ,
        0.        ],
       [0.01401855, 0.50721395, 0.34078781, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.0034968 , 0.54442287, 0.65631855, ..., 0.        , 1.        ,
        0.        ],
       [0.00235905, 0.50997182, 0.35346802, ..., 0.        , 0.        ,
        1.        ],
       [0.01401855, 0.37667593, 0.19645562, ..., 0.        , 1.        ,
        0.        ]])

In [43]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [57]:
# Feature Engineering (Polynomial Features)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
# Model Training and Testing
models = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression': LinearRegression(),
    'Least Squares Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'SVR': SVR(),
    'Decision Tree Regression': DecisionTreeRegressor(),
    'Random Forest Regression': RandomForestRegressor(),
    'Gradient Boosting Regression': GradientBoostingRegressor(),
    'Neural Network Regression': MLPRegressor()
}

from sklearn.metrics import mean_squared_error,mean_absolute_error, r2_score
import numpy as np

# Initialize lists to store metric values
model_names = []
mae_values = []
rmse_values = []
r2_values = []

# Loop through models
for name, model in models.items():
    # Fit model and make predictions
    if name == 'Polynomial Regression':
        model.fit(X_train_poly, y_train)
        y_pred = model.predict(X_test_poly)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    # Calculate RMSE
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate RMSE
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Calculate R^2 score
    r2 = r2_score(y_test, y_pred)

    # Append metric values to lists
    model_names.append(name)
    mae_values.append(mae)
    rmse_values.append(rmse)
    r2_values.append(r2)

Metric_df=pd.DataFrame({'Model': model_names, 'MAE':mae_values,'RMSE': rmse_values,'R^2 Score': r2_values})
Metric_df

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,Model,MAE,RMSE,R^2 Score
0,Linear Regression,27.800141,37.484396,0.619513
1,Polynomial Regression,15.109841,27.329081,0.797750
2,Least Squares Regression,27.800141,37.484396,0.619513
3,Ridge Regression,28.210739,37.969082,0.609610
4,Lasso Regression,28.599468,38.665520,0.595157
5,ElasticNet Regression,48.486790,59.132880,0.053116
6,SVR,45.430946,55.681653,0.160418
7,Decision Tree Regression,33.473268,43.537958,0.486696
8,Random Forest Regression,21.854628,30.691611,0.744919
9,Gradient Boosting Regression,18.666471,27.987805,0.787883
